# [LEGALST-190] Lab 4/12: Morality and Sentiment Analysis

This lab will cover morality and sentiment analysis using the *Moral Foundations Theory* with dictionary-based analysis, connecting to topic modeling and classifications ideas from previous labs.

### Table of Contents
[The Data](#section data)<br>
[Goal and Question](#section goal)<br>
1 - [Text Pre-processing](#section 1)<br>
2 - [Polarity](#section 2)<br>
3 - [Moral Foundations Theory](#section 3)<br>
4 - [Non-negative matrix factorization](#section 4)<br>

**Dependencies:**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import json

from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import nltk
from nltk.stem.snowball import SnowballStemmer

import seaborn as sns

!pip install textblob
from textblob import TextBlob

----
## The Data<a id='section data'></a>

For this lab, we'll use the Old Bailey dataset, something you all should be familiar with now. The size of the dataset is also rather large so we will compare two year-long periods, one from before 1827 and one after. Read the question to better understand why we look at 1827.

## Goal and Question<a id='section goal'></a>

The goal of today's lab is to explore sentiment analysis with three different approaches – [polarity scoring](#section 2), [topic-specific dictionary methods](#section 3), and [topic modeling](#section 4).

We'll look at sentiment in the context of the following question: 

**Did the way judges, prosecutors, and witnesses talk about moral culpability change after the Bloody Code was mostly repealed in 1827 (at the leading edge of a wave of legal reform in England)?**

*Note: this is a question that could encompass an entire research project. Today's lab uses a very small subset of data due to datahub memory limitations, and skips over many of the steps needed for truly robust conclusions. *

Something to think about: What are some things you would need to consider before answering this question?

----

## Section 1: Text Pre-processing<a id='section 1'></a>

### Before we start
This dataset we are about to look at is incredibly large, so to avoid crashing our datahub kernel, we only consider two years: 1822 and 1832. These two years were chosen as periods that were equally far from 1827 (when the Bloody Code was mostly repealed), while not being so far from each other that we'd expect to see major language usage change due only to time.


----

### Getting started

Let's get working with the data.

In [ ]:
# contains Old Bailey trial data from 1822 and 1832
old_bailey = pd.read_csv('data/obc_1822_1832.csv', index_col='trial_id')
# select only the columns we need for this lab 
old_bailey = old_bailey.loc[:, ['year', 'transcript']]
old_bailey.head()

Awesome! We now have data we can work with. Before we start anything, we must clean the text!

Just to review, we want to process our text by:<br>
1) Lowercasing the words<br>
2) Cleaning up punctuation<br>
3) Splitting into individual words<br>
4) Stemming the word tokens<br>

For the sake of time (and to get to the good stuff), we've provided the pre-processing code below. This a big data set, so the code will take up to a minute to run.

In [ ]:
# pre-process the data
lower_cased = old_bailey['transcript'].str.lower()
punct_re = r'[^\w\s]'
lower_no_punc = lower_cased.str.replace(punct_re, ' ')
tokens = lower_no_punc.str.split()
old_bailey['tokens'] = tokens
stemmer = SnowballStemmer('english')
stem_lists = []

for token_list in old_bailey['tokens']:
    stem_lists.append([stemmer.stem(wd) for wd in token_list])

old_bailey['stemmed_tokens'] = stem_lists


old_bailey.head()

----

## Section 2: Polarity <a id='section 2'></a>

One way to measure the tone of a text is to look at the text **polarity**: a measure of how positive or negative it is perceived to be. For example, a sentence like "I love Berkeley!" would be considered positive, while a sentence like "Stanford is terrible!" would be negative. And, because polarity is represented as a scale, some words have stronger positive or negative sentiment than others- "I like data science" is positive, but not as positive as "I love data science."

We will use the [TextBlob](https://textblob.readthedocs.io/en/dev/quickstart.html#sentiment-analysis) tools to analyze the sentiment of Old Bailey. TextBlob provides access to many common text-processing operations, and includes a  lexicon and rule-based sentiment analysis tool.

A TextBlob is created around string of text:

In [ ]:
# creates a sentiment analyzer
blob = TextBlob("This is a super exciting, totally awesome test sentence.")
blob

We can access the sentiment by using `.sentiment`.

In [ ]:
blob.sentiment

`sentiment` returns two values: the **polarity** and the **subjectivity**. The polarity ranges between -1 and 1 where -1 is a very negative text and 1 is a very positive text. Subjectivity ranges between 0 and 1 where 0 is a very objective text and 1 is a very subjective text (i.e. one that can be interpreted many different ways). You can get the polarity by using `.polarity`.

In [ ]:
blob.sentiment.polarity

Polarity is calculated fairly simply: TextBlob accesses a dictionary of words that have been assigned polarity and subjectivity scores, looks up each word in the given text, and averages over the sentence. It also employs a few rules, such as changing the polarity of a word that comes after a negation.

In [ ]:
happy = TextBlob('Happy')
print(happy.sentiment.polarity)

negation = TextBlob('Not')
print(negation.sentiment.polarity)

negated_happy = TextBlob('Not happy')
print(negated_happy.sentiment.polarity)

**QUESTION:** Try calculating the polarity scores of a few of your own sentences in the cell below.

In [ ]:
# test the polarity scoring for different sentences
my_blob = ...
...

Next, we want to get the average polarity for each transcript. 

**EXERCISE:** define a function that will take in a string of text and return the polarity of that text.

In [ ]:
def get_polarity(text):
    """Return the polarity of TEXT"""
    ...
    return ...

**EXERCISE**: Using `.apply` and your `get_polarity` function, get the polarity of every transcript in the Old Bailey data.

In [ ]:
polarities = ...

In [ ]:
# add the polarities as a column
old_bailey['polarity'] = polarities
old_bailey.head()

**QUESTION:** 
- What was the most negative transcript/transcripts?
- What was the most positive transcript/transcripts?


In [ ]:
# find the transcript with the highest polarity
most_pos = ...
most_pos

In [ ]:
# find the transcript with the lowest polarity
most_neg = ...
most_neg

**EXERCISE:** Let's take a look at violin plots of these two datasets to better compare how the average compound polarity is distributed for each of the two years, before and after 1827.

To show both years at once, it's easiest to use the Seaborn (abbreviated as `sns`) visualization library function. `y` is set to the name of the variable (a string) whose distributions we want to see. `x` is set to the name of the variable (also a string)that we want to compare distributions for . `data` is set to the dataframe (not a string) with all the values. 

In [ ]:
# uncomment the next line and fill in the code to create the violin plots
#sns.violinplot(x=..., y=..., data=...)

**QUESTION:** What does this plot show us? 

What are some advantages to using polarity as a way to measure moral tone? What are some issues with this approach? Consider also how these answers might change for a different data set.

*Write your answer here.*

----
## Section 3: Moral Foundations Theory<a id='section 3'></a>

Another approach is to create specialized dictionaries containing specific words of interest to try to analyze sentiment from a particular angle (i.e. use a **dictionary method**). One set of researchers did just that from the perspective of [Moral Foundations Theory](http://moralfoundations.org/). We will now use it to see if we can understand more about the moral tone of Old Bailey transcripts than by using general polarity. You should be doing something like this for your homework. We will be using a provided moral foundations dictionary.

In [ ]:
with open('data/haidt_dict.json') as json_data:
    mft_dict = json.load(json_data)

Moral Foundations Theory posits that there are five (with an occasional sixth) innane, universal psychological foundations of morality, and that those foundations shape human cultures and institutions (including legal). The keys of the dictionary correspond to the five foundations.

In [ ]:
#look at the keys of the dictionary provided
keys = mft_dict.keys()
list(keys)

And the values of the dictionary are lists of words associated with each foundation.

In [ ]:
mft_dict[list(keys)[0]] #one example of the values provided for the first key

### Calculating Percentages 

In this approach, we'll use the frequency of Moral Foundations-related words as a measure of how the transcripts talk about morality and see if there's a difference between pre- and post-1827 trends. 

As a first step, we need to know the total number of words in each transcript. 

**EXERCISE:** Add a column to `old_bailey` with the number of words corresponding to each transcript.

In [ ]:
# create a new column called 'total_words'
old_bailey['total_words'] = ...
old_bailey.head()

Next, we need to calculate the number of matches to entries in our dictionary for each foundation for each speech.

Run the next cell to add six new columns to `old_bailey`, one per foundation, that show the number of word matches. This cell will also likely take some time to run (no more than a minute). Note that by now, you have the skills to write all the code in the next cell- we're just giving it to you because it's long, fiddly, and writing nested for-loops is not the focus of this lab. Make sure you know what it does before you move on, though.

In [ ]:
# Will take a bit of time to run due to the large size.

# do the following code for each foundation
for foundation in mft_dict.keys():
    # create a new, empty column
    num_match_words = np.zeros(len(old_bailey))
    stems = mft_dict[foundation]
    
    # do the following code for each foundation word
    for stem in stems:
        # find related word matches
        wd_count = np.array([sum([wd == stem for wd in transcript])for transcript in old_bailey['stemmed_tokens']])
        # add the number of matches to the total
        num_match_words += wd_count
        
    # create a new column for each foundation with the number of related words per transcript
    old_bailey[foundation] = num_match_words

old_bailey.head()

**EXERCISE:** The columns for each foundation currently contain the number of words related to that foundation for each of the trials. Calculate the *percentage* of foundation words per trial by dividing the number of matched words by the number of total words and multiplying by 100.

In [ ]:
# do this for each foundation column
for foundation in mft_dict.keys():
    old_bailey[foundation] = ...

old_bailey.head()

Let's compare the average percentage of foundation words per transcript for the two dates, 1822, and 1832.

**EXERCISE**: Create a dataframe that only has columns for the five foundations plus the year. Then, use the pandas dataframe function `groupby` to group rows by the year, and call the `mean` function on the `groupby` output to get the averages for each foundation.

In [ ]:
# the names of the columns we want to keep
mft_columns = ['authority/subversion', 'care/harm', 'fairness/cheating', 'loyalty/betrayal',
 'sanctity/degradation', 'year']

# create a data frame with only the above columns included
mft_df = ...

# groups the rows of mft_df by year, then take the mean
foundation_avgs = ...

foundation_avgs

Next, create a bar graph. The simplest way is to call `.plot.barh()` on your dataframe of the averages. 

Also try calling `.transpose()` on your averages dataframe, then making a bar graph of that. The transpose function flips the rows and columns and can make it easier to compare the percentages.

In [ ]:
# create a bar graph
...

**QUESTION:** What do you see from the bar graphs you created? 

Why would this be a good approach to answering the question of how talk about morality changed between these two periods? What are some limitations of this approach (Hint: look at the values on the graphs you calculated, and remember: these are *percentages*, not proportions)?

*Write your answer here.*

----
## Section 4: Non-negative matrix factorization<a id='section 4'></a>

In this section, you can get an idea of sentiment using topic modeling algorithms, something you touched on in the 4/10 lab earlier this week, to help look for patterns.

On Tuesday, you explored Latent Dirichlet Allocation (LDA) in gensim to look for topics in a corpus. Non-negative matrix factorization (NMF), not included in gensim, is another such way to look for topics in unstructured text data. The two methods differ in what kinds of math they use 'under the hood': LDA relies on probabilistic graphical modeling, while NMF uses linear algebra. 

We want to generate the topics found for 1822 and 1832 trials, look for topics related to tone or morality, and see if there's a difference between the two.

Run the cell below to make two lists: one list of the trial transcripts for each year.

In [ ]:
# trial transcripts for 1822
transcripts_1822 = old_bailey[old_bailey['year'] == 1822]['transcript']

# trial transcripts for 1832
transcripts_1832 = old_bailey[old_bailey['year'] == 1832]['transcript']

We'll start by looking at 1822. The following cell creates the tfidf vectorizer, fits the text data, and assigns the list of feature name (i.e. the words in the document) to `tfidf_feature_names_1822`.

Check out the [documentation for TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) if you need a refresher on what it does.

In [ ]:
# create the vectorizer
tfidf_vectorizer_1822 = TfidfVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words='english')
# fit the data
tfidf_1822 = tfidf_vectorizer_1822.fit_transform(transcripts_1822)
# get the feature names
tfidf_feature_names_1822 = tfidf_vectorizer_1822.get_feature_names()

**EXERCISE:** Create the TfidfVectorizer, fit_transform the data, and get the feature names for 1832.

In [ ]:
# create the vectorizer
tfidf_vectorizer_1832 = ...
# fit the data
tfidf_1832 = ...
# get the feature names
tfidf_feature_names_1832 = ...

As mentioned previously the algorithms are not able to automatically determine the number of topics and this value must be set when running the algorithm. Initialising NMF with ‘nndsvd’ rather than random initialisation improves the time it takes for NMF to converge.`random_state` gives the seed for the random number generator to use: this lets us reproduce our results in the future.


In [ ]:
num_topics = 20
# Run NMF for 1822
nmf_1822 = NMF(n_components=num_topics, random_state=1, init='nndsvd').fit(tfidf_1822)

**EXERCISE:** Run NMF using `num_topics` for the number of components on the data from 1832.

In [ ]:
# Run NMF for 1832
nmf_1832 = ...

We've provided you the function to display the topics shown by the NMF.

In [ ]:
def display_topics(model, feature_names, num_top_words):
    """Displays NUM_TOP_WORDS topics for MODEL """
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-num_top_words - 1:-1]]))

# the number of words to display per topic
num_top_words = 10

# display the topics for 1822
display_topics(nmf_1822, tfidf_feature_names_1822, num_top_words)


In [ ]:
# display the topics for 1832
display_topics(nmf_1832, tfidf_feature_names_1832, num_top_words)

As in LDA, it often takes some hyperparameter tuning before you get a coherent set of topics. Go back and tune the parameters for tfidf and NMF to see if you can get topics that show something about the moral sentiment of the transcripts. `num_topics` usually makes the most dramatic difference.

Once you've compared several sets of topics, answer the next question:

**QUESTION:** What did the best set of topics you found tell you about the tone of the documents for the two periods? Why might this approach be a good way to study sentiment? What are some issues with this approach?

*Write your answer here.*

----
Fantastic! Now you know how to approach sentiment analysis several ways using general sentiment analysis with `VADER`, Moral Foundations Theory, and Non-negative matrix factorization.

Ultimately, there were limitations to all of these methods. In your homework, you'll explore how to generate your own dictionary to try to overcome some of these limitations.

----

## Bibliography


 - Moral Foundations Theory background and dictionary: *Jesse Graham and Jonathan Haidt http://moralfoundations.org/*
 - Moral Foundations analysis code adapted from *https://github.com/ds-modules/XRHETOR-R1A/blob/master/02-Moral-Foundations-Analysis/02-Moral-Foundations-Analysis.ipynb*
 - NMF code and explanation adapted from *Aneesha Bakharia. 'Topic Modeling With Scikit-Learn.' https://medium.com/mlreview/topic-modeling-with-scikit-learn-e80d33668730*

----
Notebook developed by: Gibson Chu, Keeley Takimoto

Data Science Modules: http://data.berkeley.edu/education/modules